Ceno Scraper


##Lista kroków  "manulanych"
1. Wejście na stronę produktu / z opiniami o produkcie ceno.pl
2.Dla każdej opini na stronie 
    2.1.Skopjuj opinię
    2.2.Wklej opinie do edytora tekstu
3.Przejdź do kolejnej strony z opiniami
4.Powtarzaj kroki 2-3 dopóki są strony z opiniami

#Lista kroków scrapera
1.Wysłanie żądanie dostępu do zasobu - strona z opiniami o produkcie

## Struktura opini w serwisie Ceno.pl

|składowa|nazwa|selektor|
|--------|-----|--------|
|identyfikator opinii|opinion_id|["data-entry-id"]|
|autor|author|span.user-post__author-name|
|rekomendacja|recommednation|span.user-post__recomendation > em|
|treść opini|content|div.user-post__text|
|listę zalet|pros|div.review-feature__title--positivies ~ div.review-feature__item|
|listę wad|cons|div.review-feature__title--negatives ~ div.review-feature__item|
|liczbę gwiazdek|stars|span.user-post__score-count|
|data wystawienia opinii|post_date|span.user-post__published >time:nth-child(1)["date-time"]|
|data zakupu produktu|purchase_date|pan.user-post__published >time:nth-child(2)["date-time"]|
|ile osób uznało opinię za przydatną|useful|button.vote-yes > span|
|ile osób uznało opinię za nieprzydatną|useless|button.vote-no > span|
 treść opinii


63717975
124893467
49729622
84514582
61161229

1.Import bibliotek


In [18]:
import os
import json
import requests
from bs4 import BeautifulSoup

2.Wysłanie żądanie dostępu do zasobu(request)


In [19]:
product_code = "63717975"
url=f"https://www.ceneo.pl/{product_code}#tab=reviews"
response = requests.get(url)

3.Wydobycie z kodu HTML fragmetów odpowiadajacych  poszczególnym  opinon

In [20]:
page = BeautifulSoup(response.text, "html.parser")
opinions= page.select("div.js_product-review")


4.Wydobycie z kodu opini jej składowych dla każdej opinii na stronie 

In [23]:
all_opinions = []
for opinion in opinions:
    single_opinion = {
        'opinion_id': opinion["data-entry-id"],
        'author': opinion.select_one("span.user-post__author-name").text.strip(),
        'recommendation': opinion.select_one("span.user-post__recommendation > em").text.strip() if opinion.select_one("span.user-post__recommendation > em") else None,
        'stars': opinion.select_one("span.user-post__score-count").text.strip(),
        'content': opinion.select_one("div.user-post__text").text.strip(),
        'pros': [p.text.strip() for p in opinion.select("div.review-feature__title--positives ~ div.review-feature__item")],
        'cons': [c.text.strip() for c in opinion.select("div.review-feature__title--negatives ~ div.review-feature__item")],
        'pros_date': opinion.select_one("span.user-post__published > time:nth-child(1)").get("date-time", None),
        'purchase_date': opinion.select_one("span.user-post__published > time:nth-child(2)").get("date-time", None),
        'useful': opinion.select_one("button.vote-yes > span").text.strip(),
        'useless': opinion.select_one("button.vote-no > span").text.strip(),
    }
    all_opinions.append(single_opinion)
    print(single_opinion)




{'opinion_id': '16455702', 'author': 'p...m', 'recommendation': None, 'stars': '5/5', 'content': 'Mała zgrabna drukarka do podstawowych domowych potrzeb. Jakość wydruku bardzo dobra. Drukuje szybko, około 20str/min. Łączność przez Wi-Fi genialna. Bez problemu drukuje z komputera czy telefonu. Nie używam na smartphonie aplikacji Brother wiec na ten temat się nie wypowiem ale przez aplikacje Printer Share działa genialnie. Jest bardzo mała więc bez problemu weszła do małej szafki. Przy zamkniętej klapie wydrukowane kartki wysuwają się przez szczelinę dzięki czemu klapa nie musi być uniesiona i się niepotrzebnie kurzyć. Na minus brak kabla w zestawie, który jest niezbędny aby połączyć drukarkę z siecią Wi-Fi. Poza tym drukarka sprawuje się świetnie.\nPlusy:\n-szybkie drukowanie\n-bardzo dobra jakość druku\n-mała zgrabna wielkość drukarki\n-łączność przez Wi-Fi\n-prosta obsługa\nMinusy\n-brak kabla w zestawie', 'pros': ['cena', 'głośność pracy', 'jakość wydruku', 'szybkość wydruku'], 'cons

5.Zapis opini do pliku JSON


In [24]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")
jf = open(f"{product_code}.json","w",encoding="UTF-8")
json.dump(all_opinions,jf, indent=4, ensure_ascii=False)
jf.close()